## Import and setup

In [1]:
#base
import pandas as pd
import numpy as np
import re

In [2]:
#text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Data Ingestion

In [4]:
data = pd.read_csv('https://www.dati.lombardia.it/api/views/xwm8-byxq/rows.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131718 entries, 0 to 131717
Data columns (total 83 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   ID risposta                                       131718 non-null  int64  
 1   Data invio                                        131718 non-null  object 
 2   Anno                                              131718 non-null  int64  
 3   Mese                                              131718 non-null  int64  
 4   Piattaforma usata per l'adesione al bando         131707 non-null  object 
 5   Codice piattaforma usata per l'adesione al bando  131707 non-null  object 
 6   Codice bando                                      131714 non-null  object 
 7   Titolo bando                                      131718 non-null  object 
 8   Fase                                              131718 non-null  object 
 9   Link

,ID risposta,Data invio,Anno,Mese,Piattaforma usata per l'adesione al bando,Codice piattaforma usata per l'adesione al bando,Codice bando,Titolo bando,Fase,Link_report,A 1,A 2.1,A 2.2,A 2.3,A 2.4,A 2.5,A 2.6,A 2.7,A 2.8,A 3,A 3a.1,A 3a.2,A 3a.3,A 3a.4,A 3a.5,A 3a.6,A 4,A 5.1,A 5.2,A 5.3,A 5.4,A 5.5,A 5.6,A 5.7,A 5.8,A 6.1,A 6.2,A 6.3,A 6.4,A 6.5,...,A 6a.3,A 6a.4,A 7,A 7a.1,A 7a.2,A 7a.3,A 7a.4,A 7a.5,A 7b,A 7c,A 8,A 9.1,A 9.2,A 9.3,A 9.4,R 10,R 11.1,R 11.2,R 11.3,R 12.1,R 12.2,R 12.3,R 12.4,R 12.5,R 12.6,R 12a.1,R 12a.2,R 12a.3,R 12a.4,R 13,R 13a.1,R 13a.2,R 13a.3,R 13a.4,R 13a.5,R 13b,R 13c,R 14,R 14a,T 1
0,60607,27/04/2018,2018,4,SiAge,A1,RLE12018004842,Dote Scuola 2018,Adesione,http://supportsiss.lispa.it/opendata/cs/201805...,5.0,NO,NO,SI,NO,NO,NO,NO,NaN,4.0,NO,SI,NO,NO,NO,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,NO,NO,NO,SI,NO,...,NaN,5.0,SI,SI,NO,NO,NO,NO,SI,5.0,5.0,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.40
1,89537,19/10/2018,2018,10,SiAge,A1,RLN12018005682,Dote sport 2018,Adesione,http://supportsiss.lispa.it/opendata/cs/201810...,5.0,NO,NO,NO,NO,SI,NO,NO,NaN,5.0,NO,NO,NO,SI,NO,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,NO,NO,NO,SI,NO,...,NaN,5.0,SI,NO,NO,NO,NO,SI,SI,5.0,5.0,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244.37
2,227440,05/10/2020,2020,10,Bandi online,A1,RLE12020011322,Dote Scuola 2020 BUONO SCUOLA,Adesione,http://supportsiss.lispa.it/opendata/cs/202011...,5.0,NO,NO,NO,NO,NO,NO,NO,CONTROLLO SEMPRE IL SITO DELLA REGIONE LOMBARDIA,5.0,NO,NO,NO,NO,SI,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,SI,...,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.39
3,222007,10/09/2020,2020,9,Bandi online,A1,RLA12020013962,Concorso RegioneProvincie CAT C informatico,Adesione,http://supportsiss.lispa.it/opendata/cs/202009...,5.0,NO,NO,NO,NO,NO,NO,NO,DAL SITO DI REGIONE LOMBARDIA,2.0,NO,NO,NO,SI,SI,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,SI,...,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227.58
4,193611,23/05/2020,2020,5,Bandi online,A1,RLE12020009702,DOTE SCUOLA - Materiale Didattico a.s. 2020-20...,Adesione,http://supportsiss.lispa.it/opendata/cs/202006...,3.0,NO,NO,NO,NO,NO,NO,SI,NaN,4.0,NO,NO,NO,NO,NO,NULLA,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,SI,...,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.58


## Data cleaning

In [5]:
data_f = data[-data['R 14a'].isna()]
data_f.info()
data_f.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2186 entries, 17 to 131715
Data columns (total 83 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   ID risposta                                       2186 non-null   int64  
 1   Data invio                                        2186 non-null   object 
 2   Anno                                              2186 non-null   int64  
 3   Mese                                              2186 non-null   int64  
 4   Piattaforma usata per l'adesione al bando         2185 non-null   object 
 5   Codice piattaforma usata per l'adesione al bando  2185 non-null   object 
 6   Codice bando                                      2186 non-null   object 
 7   Titolo bando                                      2186 non-null   object 
 8   Fase                                              2186 non-null   object 
 9   Link_report     

,ID risposta,Data invio,Anno,Mese,Piattaforma usata per l'adesione al bando,Codice piattaforma usata per l'adesione al bando,Codice bando,Titolo bando,Fase,Link_report,A 1,A 2.1,A 2.2,A 2.3,A 2.4,A 2.5,A 2.6,A 2.7,A 2.8,A 3,A 3a.1,A 3a.2,A 3a.3,A 3a.4,A 3a.5,A 3a.6,A 4,A 5.1,A 5.2,A 5.3,A 5.4,A 5.5,A 5.6,A 5.7,A 5.8,A 6.1,A 6.2,A 6.3,A 6.4,A 6.5,...,A 6a.3,A 6a.4,A 7,A 7a.1,A 7a.2,A 7a.3,A 7a.4,A 7a.5,A 7b,A 7c,A 8,A 9.1,A 9.2,A 9.3,A 9.4,R 10,R 11.1,R 11.2,R 11.3,R 12.1,R 12.2,R 12.3,R 12.4,R 12.5,R 12.6,R 12a.1,R 12a.2,R 12a.3,R 12a.4,R 13,R 13a.1,R 13a.2,R 13a.3,R 13a.4,R 13a.5,R 13b,R 13c,R 14,R 14a,T 1
17,148246,04/12/2019,2019,12,Bandi online,A1,RLO12019006542,FABER I Ed,Rendicontazione,http://supportsiss.lispa.it/opendata/cs/201912...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NO,SI,NO,SI,NO,NO,NO,NO,NO,4.0,NaN,NaN,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,CI SONO TROPPE INFORMAZIONI DA RIPETERE NEI MO...,197.24
89,255807,20/01/2021,2021,1,Bandi online,A1,RLN12020011162,E di nuovo sport,Rendicontazione,http://supportsiss.lispa.it/opendata/cs/202103...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NO,SI,NO,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,SI,NO,SI,NO,NO,NO,SI,5.0,5.0,DI FACILE E COMPRENSIBILE UTILIZZO,195.09
99,238123,16/11/2020,2020,11,Bandi online,A1,RLT12019007722,Smaltimento amianto Anno 2019,Rendicontazione,http://supportsiss.lispa.it/opendata/cs/202012...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NO,SI,NO,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,NaN,SI,SI,SI,NO,NO,NO,NO,2.0,3.0,NON HO RICEVUTO RISPOSTA ALLA RICHIESTA DI ASS...,71.88
195,8100,28/04/2017,2017,4,SiAge,A1,RLL12016000790,Musei 2016,Rendicontazione,http://supportsiss.lispa.it/opendata/cs/201706...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NO,SI,NO,NO,NO,NO,SI,NO,NO,NaN,NaN,NaN,3.0,SI,SI,SI,NO,NO,NO,SI,3.0,3.0,NON TUTTI GLI AUTOMATISMI DEL SISTEMA FUNZIONA...,346.25
233,198665,03/06/2020,2020,6,Bandi online,A1,RLT12019008702,Rinnova Autoveicoli,Rendicontazione,http://supportsiss.lispa.it/opendata/cs/202006...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NO,SI,NO,NO,NO,NO,SI,NO,NO,NaN,NaN,NaN,3.0,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,"NON CHIARI I PERCORSI PER OPERARE, IN PARTICOL...",332.05


r14 -> punteggio \\
r 14a -> commento

In [6]:
columns = ['Titolo bando', 'Piattaforma usata per l\'adesione al bando', 'R 14', 'R 14a']
data_f = data_f[columns]

In [7]:
data_f.columns = ['bando', 'piattaforma', 'punteggio', 'commento']
data_f.info()
data_f.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2186 entries, 17 to 131715
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bando        2186 non-null   object 
 1   piattaforma  2185 non-null   object 
 2   punteggio    2186 non-null   float64
 3   commento     2186 non-null   object 
dtypes: float64(1), object(3)
memory usage: 85.4+ KB


,bando,piattaforma,punteggio,commento
17,FABER I Ed,Bandi online,3.0,CI SONO TROPPE INFORMAZIONI DA RIPETERE NEI MO...
89,E di nuovo sport,Bandi online,5.0,DI FACILE E COMPRENSIBILE UTILIZZO
99,Smaltimento amianto Anno 2019,Bandi online,3.0,NON HO RICEVUTO RISPOSTA ALLA RICHIESTA DI ASS...
195,Musei 2016,SiAge,3.0,NON TUTTI GLI AUTOMATISMI DEL SISTEMA FUNZIONA...
233,Rinnova Autoveicoli,Bandi online,3.0,"NON CHIARI I PERCORSI PER OPERARE, IN PARTICOL..."


In [8]:
len(data_f['bando'].unique())

90

In [9]:
len(data_f['piattaforma'].unique())

6

## Text cleaning

In [10]:
for col in ['bando', 'piattaforma', 'commento']:
  data_f[col] = data_f[col].str.lower()
data_f.head()

,bando,piattaforma,punteggio,commento
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...
233,rinnova autoveicoli,bandi online,3.0,"non chiari i percorsi per operare, in particol..."


In [11]:
#remove punctuation
def remove_punctuation(s):
    s = re.sub(r'[^\w\s]', ' ', str(s).lower().strip())
    return s
data_f['commento'] = data_f['commento'].apply(remove_punctuation)

In [12]:
#tokenization
data_f['commento_p'] = data_f['commento'].apply(word_tokenize)
data_f.head()

,bando,piattaforma,punteggio,commento,commento_p
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...,"[ci, sono, troppe, informazioni, da, ripetere,..."
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo,"[di, facile, e, comprensibile, utilizzo]"
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...,"[non, ho, ricevuto, risposta, alla, richiesta,..."
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...,"[non, tutti, gli, automatismi, del, sistema, f..."
233,rinnova autoveicoli,bandi online,3.0,non chiari i percorsi per operare in particol...,"[non, chiari, i, percorsi, per, operare, in, p..."


In [13]:
#remove stopwords
stopWords = stopwords.words('italian')
data_f['commento_p'] = data_f['commento_p'].apply(lambda x: [item for item in x if item == "non" or item not in stopWords])
data_f.head()

,bando,piattaforma,punteggio,commento,commento_p
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...,"[troppe, informazioni, ripetere, moduli, facil..."
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo,"[facile, comprensibile, utilizzo]"
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...,"[non, ricevuto, risposta, richiesta, assitenza]"
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...,"[non, automatismi, sistema, funzionano, stati,..."
233,rinnova autoveicoli,bandi online,3.0,non chiari i percorsi per operare in particol...,"[non, chiari, percorsi, operare, particolare, ..."


In [14]:
def negate(words):

    negation = False
    result = []

    for word in words:
        stripped = word.lower()
        negated = "non_" + stripped if negation and stripped != "non" else stripped
        result.append(negated)

        if any(neg == word for neg in ["non"]):
            negation = True

    return result

In [15]:
#handling negation
data_f['commento_p'] = data_f['commento_p'].apply(negate)

In [16]:
data_f.head()

,bando,piattaforma,punteggio,commento,commento_p
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...,"[troppe, informazioni, ripetere, moduli, facil..."
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo,"[facile, comprensibile, utilizzo]"
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...,"[non, non_ricevuto, non_risposta, non_richiest..."
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...,"[non, non_automatismi, non_sistema, non_funzio..."
233,rinnova autoveicoli,bandi online,3.0,non chiari i percorsi per operare in particol...,"[non, non_chiari, non_percorsi, non_operare, n..."


In [17]:
def to_string(tokens):
  return " ".join(token for token in tokens)
data_f['commento_p'] = data_f['commento_p'].apply(to_string)
data_f.head()

,bando,piattaforma,punteggio,commento,commento_p
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...,troppe informazioni ripetere moduli facilmente...
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo,facile comprensibile utilizzo
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...,non non_ricevuto non_risposta non_richiesta no...
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...,non non_automatismi non_sistema non_funzionano...
233,rinnova autoveicoli,bandi online,3.0,non chiari i percorsi per operare in particol...,non non_chiari non_percorsi non_operare non_pa...


In [18]:
#0 -> negative
#1 -> positive
data_f['target'] = data_f['punteggio'].apply(lambda x: 1 if x > 3 else 0)

In [19]:
data_f

,bando,piattaforma,punteggio,commento,commento_p,target
17,faber i ed,bandi online,3.0,ci sono troppe informazioni da ripetere nei mo...,troppe informazioni ripetere moduli facilmente...,0
89,e di nuovo sport,bandi online,5.0,di facile e comprensibile utilizzo,facile comprensibile utilizzo,1
99,smaltimento amianto anno 2019,bandi online,3.0,non ho ricevuto risposta alla richiesta di ass...,non non_ricevuto non_risposta non_richiesta no...,0
195,musei 2016,siage,3.0,non tutti gli automatismi del sistema funziona...,non non_automatismi non_sistema non_funzionano...,0
233,rinnova autoveicoli,bandi online,3.0,non chiari i percorsi per operare in particol...,non non_chiari non_percorsi non_operare non_pa...,0
...,...,...,...,...,...,...
131176,sicurezza_urbana_lotto_b_2017,siage,5.0,e un sistema agile e rapido per interagire ...,sistema agile rapido interagire non non_richie...,1
131177,smaltimento amianto anno 2019,bandi online,3.0,molto complicato senza l ausilio di un espert...,molto complicato senza ausilio esperto compute...,0
131346,bando accumulo anno 2017,bandi online,4.0,tutto ok,ok,1
131578,asd 2018,bandi online,4.0,buono nel complesso migliorabile attraverso u...,buono complesso migliorabile attraverso puntua...,1


In [22]:
data_f[["bando",	"piattaforma", "commento_p",	"target"]].to_csv('cleaned.csv', index=None)